In [15]:
import abtem
import ase
import numpy as np
import py4DSTEM
from scipy.ndimage import rotate

abtem.config.set({"dask.lazy":False});

In [3]:
srtio3 = ase.io.read("data/SrTiO3.cif")*(6,6,2)
gpts = (192,192)
scan_gpts = (32,32)
energy = 200e3
semiangle_cutoff = 20

In [4]:
potential = abtem.Potential(
    srtio3,
    gpts=gpts,
    slice_thickness=srtio3.cell[2,2]/8,
    projection='finite'
).build(
)

In [5]:
grid_scan = abtem.GridScan(
    start=(0,0),
    end=np.array(potential.extent)/6,
    gpts=scan_gpts,
)

In [6]:
detector = abtem.PixelatedDetector(max_angle=None)

In [7]:
probe_in_focus = abtem.Probe(
    semiangle_cutoff=semiangle_cutoff,
    energy=energy
).match_grid(
    potential
)

In [8]:
measurement_in_focus = probe_in_focus.scan(
    potential,
    grid_scan,
    detector
)

In [9]:
probe_defocus = abtem.Probe(
    semiangle_cutoff=semiangle_cutoff,
    energy=energy,
    C10=-100,
).match_grid(
    potential
)

In [10]:
measurement_defocus = probe_defocus.scan(
    potential,
    grid_scan,
    detector
)

In [11]:
probe_aberrated = abtem.Probe(
    semiangle_cutoff=semiangle_cutoff,
    energy=energy,
    C10=-100,C23=10000,phi23=np.deg2rad(27.5),
).match_grid(
    potential
)

In [12]:
measurement_aberrated = probe_aberrated.scan(
    potential,
    grid_scan,
    detector
)

In [13]:
def crop_rotate_noise(measurement, angle_deg = 15, electrons_per_area = 1e5):
    """ """
    array = measurement.array[:,:,gpts[0]//4:-gpts[0]//4,gpts[1]//4:-gpts[1]//4]
    array = rotate(array,angle_deg,reshape=False,order=1,axes=(-1,-2)).clip(0)

    electrons_per_probe = np.prod(grid_scan.sampling) * electrons_per_area
    array = np.random.poisson(array * electrons_per_probe).clip(0).astype(np.uint16)
    return array
    

In [14]:
data_in_focus = crop_rotate_noise(measurement_in_focus)
data_defocus = crop_rotate_noise(measurement_defocus)
data_aberrated = crop_rotate_noise(measurement_aberrated)

In [19]:
dc_in_focus = py4DSTEM.DataCube(data_in_focus)
dc_in_focus.calibration.set_Q_pixel_size(measurement_in_focus.angular_sampling[0])
dc_in_focus.calibration.set_R_pixel_size(grid_scan.sampling[0])
dc_in_focus.calibration.set_Q_pixel_units('mrad')
dc_in_focus.calibration.set_R_pixel_units('A')
dc_in_focus.calibration

Calibration( A Metadata instance called 'calibration', containing the following fields:

             Q_pixel_size:    1.0595062907655965
             R_pixel_size:    0.12328531250000001
             Q_pixel_units:   mrad
             R_pixel_units:   A
             QR_flip:         False
)

In [22]:
dc_defocus = py4DSTEM.DataCube(data_defocus,calibration=dc_in_focus.calibration.copy())
dc_aberrated = py4DSTEM.DataCube(data_aberrated,calibration=dc_in_focus.calibration.copy())

In [23]:
py4DSTEM.save("data/dpc_STO_simulation_in-focus_1e5.h5",dc_in_focus,mode='o')
py4DSTEM.save("data/dpc_STO_simulation_defocus_1e5.h5",dc_defocus,mode='o')
py4DSTEM.save("data/dpc_STO_simulation_aberrated_1e5.h5",dc_aberrated,mode='o')